# Used Vehicle Price Prediction: KaggleX Skill Assessment Challenge
This work is for the participation the challenge mentioned in the title, aiming to predict used vehicle prices based on the data given.

## Dataset
We are given train.csv and test.csv, with the former (as the name suggests) having 12 features column an 1 target column of price. The test data lacks the target price column so has 12 columns.

The test data is usually large (from my experience), having about 36k rows compared to the 54k rows in the training dataset. (may make the prediction hard if the test data distribution is marginally different from training data?)

## Methodology
Off the top of my head I will approach this similar to my previous project where we follow the steps of:
1. data exploration: distribution, outliers, data types, correlation...
2. data preprocessing: data cleaning, feature engineering, train-test split
3. baseline modeling: use baseline models like decision trees, random forest & linear regression
4. model2 : build fancy model trying to beat baseline model
5. model tuning: overfit then prune? hyperparameter-tuning? monitor loss-curve? early stopping?
6. model evaluation?


# 1. Data Preparation 

## 1.1 Data Loading

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kagglex/sample_submission.csv
/kaggle/input/kagglex/train.csv
/kaggle/input/kagglex/test.csv


In [2]:
#load the train.csv into a dataframe
train_df = pd.read_csv('/kaggle/input/kagglex/train.csv')
test_df = pd.read_csv('/kaggle/input/kagglex/test.csv')

print(train_df.shape)
print(test_df.shape)

(54273, 13)
(36183, 12)


## 1.2 Data Exploration

In [3]:
train_df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [4]:
test_df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
1,54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes
2,54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes
3,54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes
4,54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes


Quick look and the data suggest some columns should be more valuable than others?
* brand
* ~model??~
* model_year
* fuel_type
* milage (need transformation?)
* ext_col (need transformation, make it simple)
* accident

engine is a mess (need transformation), will not consider first as heuristically i think it might be less important. color can be important but not sure if enough. year, brand and accident should the most important.

lets check the distribution for numerical and unique value of each categorical column to further determine:

In [5]:
# check the distribution of numerical numbers
train_df.describe()

,id,model_year,milage,price
count,54273.000000,54273.000000,54273.000000,5.427300e+04
mean,27136.000000,2015.091979,72746.175667,3.921844e+04
std,15667.409917,5.588909,50469.490448,7.282634e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,13568.000000,2012.000000,32268.000000,1.550000e+04
50%,27136.000000,2016.000000,66107.000000,2.800000e+04
75%,40704.000000,2019.000000,102000.000000,4.500000e+04
max,54272.000000,2024.000000,405000.000000,2.954083e+06


In [6]:
# check the distribution of numerical numbers
test_df.describe()

,id,model_year,milage
count,36183.000000,36183.000000,36183.000000
mean,72364.000000,2015.063953,72479.266755
std,10445.276732,5.589336,50714.968252
min,54273.000000,1974.000000,100.000000
25%,63318.500000,2012.000000,31681.000000
50%,72364.000000,2016.000000,65680.000000
75%,81409.500000,2019.000000,102000.000000
max,90455.000000,2024.000000,405000.000000


The distribution of test data falls right in line with that of the training data's, which should be really good for producing accurate predictions.

In [7]:
# check unique values of categorical data
print("columns and respective unique values:")
print("brands:", train_df.brand.unique())
# print("model:", train_df.model.unique())
print("fuel_type:", train_df.fuel_type.unique())
print("ext_col:", train_df.ext_col.unique())
print("clean_title:", train_df.clean_title.unique())
print("accident:", train_df.accident.unique())

columns and respective unique values:
brands: ['Ford' 'BMW' 'Jaguar' 'Pontiac' 'Acura' 'Audi' 'GMC' 'Maserati'
 'Chevrolet' 'Porsche' 'Mercedes-Benz' 'Tesla' 'Lexus' 'Kia' 'Lincoln'
 'Dodge' 'Volkswagen' 'Land' 'Cadillac' 'Mazda' 'RAM' 'Subaru' 'Hyundai'
 'MINI' 'Jeep' 'Honda' 'Hummer' 'Nissan' 'Toyota' 'Volvo' 'Genesis'
 'Mitsubishi' 'Buick' 'INFINITI' 'McLaren' 'Scion' 'Lamborghini' 'Bentley'
 'Suzuki' 'Ferrari' 'Alfa' 'Rolls-Royce' 'Chrysler' 'Aston' 'Rivian'
 'Lotus' 'Saturn' 'Lucid' 'Mercury' 'Maybach' 'FIAT' 'Plymouth' 'Bugatti']
fuel_type: ['Gasoline' 'Hybrid' 'E85 Flex Fuel' 'Diesel' '–' 'Plug-In Hybrid'
 'not supported']
ext_col: ['Blue' 'Black' 'Purple' 'Gray' 'White' 'Red' 'Silver' 'Summit White'
 'Platinum Quartz Metallic' 'Green' 'Orange' 'Lunar Rock'
 'Red Quartz Tintcoat' 'Beige' 'Gold' 'Jet Black Mica'
 'Delmonico Red Pearlcoat' 'Brown' 'Rich Garnet Metallic'
 'Stellar Black Metallic' 'Yellow' 'Deep Black Pearl Effect' 'Metallic'
 'Ice Silver Metallic' 'Agate Black Meta

In [8]:
# check unique values of categorical data
print("columns and respective unique values:")
print("brands:", test_df.brand.unique())
# print("model:", test_df.model.unique())
print("fuel_type:", test_df.fuel_type.unique())
print("ext_col:", test_df.ext_col.unique())
print("clean_title:", test_df.clean_title.unique())
print("accident:", test_df.accident.unique())

columns and respective unique values:
brands: ['Mercedes-Benz' 'Lexus' 'Land' 'BMW' 'Chevrolet' 'Dodge' 'Audi' 'Ford'
 'Kia' 'Toyota' 'Cadillac' 'GMC' 'Jeep' 'Mazda' 'Acura' 'INFINITI'
 'Volkswagen' 'Subaru' 'Hyundai' 'Jaguar' 'Porsche' 'Lincoln' 'Nissan'
 'RAM' 'Buick' 'Honda' 'MINI' 'Rolls-Royce' 'Genesis' 'Bentley' 'Volvo'
 'Saturn' 'Ferrari' 'Bugatti' 'Tesla' 'Pontiac' 'Hummer' 'Mitsubishi'
 'Maserati' 'Alfa' 'Scion' 'Lamborghini' 'Chrysler' 'McLaren' 'Lotus'
 'Rivian' 'Aston' 'FIAT' 'Lucid' 'Mercury' 'Suzuki' 'Saab' 'smart']
fuel_type: ['Gasoline' 'E85 Flex Fuel' 'Diesel' 'Hybrid' '–' 'Plug-In Hybrid'
 'not supported']
ext_col: ['White' 'Silver' 'Blue' 'Red' 'Black' 'Gray' 'Atomic Silver' 'Green'
 'Octane Red Pearlcoat' 'Purple' 'Diamond Black' 'Agate Black Metallic'
 '–' 'Orange' 'Polymetal Gray Metallic' 'Crystal Black Pearl'
 'Snowflake White Pearl' 'Jet Black Mica' 'Black Raven' 'Black Clearcoat'
 'Yellow' 'Metallic' 'Imperial Blue Metallic' 'Phytonic Blue Metallic'
 'Gold' 'B

lets check missing data:

In [9]:
# check missing values
print("NaN value in brand:", train_df.brand.isna().sum())
print("NaN value in model:", train_df.model.isna().sum())
print("NaN value in model_year:", train_df.model_year.isna().sum())
print("NaN value in fuel_type:", train_df.fuel_type.isna().sum())
print("'-' or 'not supported' value in fuel_type:", train_df[(train_df.fuel_type == '–') | (train_df.fuel_type == 'not supported')].shape[0])
print("NaN value in milage:", train_df.milage.isna().sum())
print("NaN value in ext_col:", train_df.ext_col.isna().sum())
print("'-' in ext_col:", train_df[train_df.ext_col == '–'].shape[0])
print("NaN value in accident:", train_df.accident.isna().sum())
print("NaN value in price:", train_df.price.isna().sum())
print("0 value in price:", train_df[(train_df.price == 0)].shape[0])

NaN value in brand: 0
NaN value in model: 0
NaN value in model_year: 0
NaN value in fuel_type: 0
'-' or 'not supported' value in fuel_type: 298
NaN value in milage: 0
NaN value in ext_col: 0
'-' in ext_col: 41
NaN value in accident: 0
NaN value in price: 0
0 value in price: 0


In [10]:
# check missing values
print("NaN value in brand:", test_df.brand.isna().sum())
print("NaN value in model:", test_df.model.isna().sum())
print("NaN value in model_year:", test_df.model_year.isna().sum())
print("NaN value in fuel_type:", test_df.fuel_type.isna().sum())
print("'-' or 'not supported' value in fuel_type:", test_df[(test_df.fuel_type == '–') | (test_df.fuel_type == 'not supported')].shape[0])
print("NaN value in milage:", test_df.milage.isna().sum())
print("NaN value in ext_col:", test_df.ext_col.isna().sum())
print("'-' in ext_col:", test_df[test_df.ext_col == '–'].shape[0])
print("NaN value in accident:", test_df.accident.isna().sum())

NaN value in brand: 0
NaN value in model: 0
NaN value in model_year: 0
NaN value in fuel_type: 0
'-' or 'not supported' value in fuel_type: 201
NaN value in milage: 0
NaN value in ext_col: 0
'-' in ext_col: 27
NaN value in accident: 0


quick thoughts upon inspection:

There are columns that are clearly useful and important:
* *brands*
* *model_year*
* *milage*
* *accident*, can be changed to 1 & 0 to indicate

There are also columns that needs work:
* *fuel_type* has some missing value & might be useful, we will drop columns with missing values & proceed, and change 'E85 Flex Fuel' to 'Gasoline' aaaand 'Plug-In Hybrid' to 'Hybrid'
* *ext_col* may be useful, but there is a few missing values & need transformation (try to convert most to simple color: white, red, black etc)

Finally there are columns deemed not significant and we will proceed without for now:
* *model* will not be used for now, a lot of work to do and seems less significant

Also it is noteworthy that the target value *price* is free of missing value or 0

# 2. ~Data Preprocessing~
## ~2.1 Data Cleaning~
~remove rows with *fuel_type* having missing values~

instead, we will allow '-' for fuel_type and ext_col below...

In [11]:
# train_df = train_df[train_df['fuel_type'] != '–']
# train_df = train_df[train_df['fuel_type'] != 'not supported']
# train_df = train_df[train_df['ext_col'] != '–']
# print("'-' or 'not supported' value in fuel_type:", train_df[(train_df.fuel_type == '–') | (train_df.fuel_type == 'not supported')].shape[0])
# print("'-' in ext_col:", train_df[train_df.ext_col == '–'].shape[0])
# train_df.shape

In [12]:
# test_df = test_df[test_df['fuel_type'] != '–']
# test_df = test_df[test_df['fuel_type'] != 'not supported']
# test_df = test_df[test_df['ext_col'] != '–']
# print("'-' or 'not supported' value in fuel_type:", test_df[(test_df.fuel_type == '–') | (test_df.fuel_type == 'not supported')].shape[0])
# print("'-' in ext_col:", test_df[test_df.ext_col == '–'].shape[0])
# test_df.shape

## 2.2 Data Transformation

### 2.2.1 Replacing values in accident & fuel_type

In [13]:
pd.set_option('future.no_silent_downcasting', True)

In [14]:
# Change accident to numerical of 1 & 0
train_df['accident'] = train_df['accident'].replace('None reported', 0 )
train_df['accident'] = train_df['accident'].replace('At least 1 accident or damage reported', 1 )
print("train accident:", train_df.accident.unique())

test_df['accident'] = test_df['accident'].replace('None reported', 0 )
test_df['accident'] = test_df['accident'].replace('At least 1 accident or damage reported', 1 )
print("test accident:", test_df.accident.unique())

train accident: [0 1]
test accident: [0 1]


In [15]:
# Change fuel_type to narrow down the types
train_df['fuel_type'] = train_df['fuel_type'].replace('E85 Flex Fuel', 'Gasoline')
train_df['fuel_type'] = train_df['fuel_type'].replace('Plug-In Hybrid', 'Hybrid')
train_df['fuel_type'] = train_df['fuel_type'].replace('not supported', '–')
print("train fuel_type:", train_df.fuel_type.unique())

test_df['fuel_type'] = test_df['fuel_type'].replace('E85 Flex Fuel', 'Gasoline')
test_df['fuel_type'] = test_df['fuel_type'].replace('Plug-In Hybrid', 'Hybrid')
test_df['fuel_type'] = test_df['fuel_type'].replace('not supported', '–')
print("test fuel_type:", test_df.fuel_type.unique())

train fuel_type: ['Gasoline' 'Hybrid' 'Diesel' '–']
test fuel_type: ['Gasoline' 'Diesel' 'Hybrid' '–']


### 2.2.2 Deal with the strings in column ext_col to make them more generic
turn weird color names into general colors (e.g. white, black, blue...)
make a new column?
then mayyybe remove the weird colors if there is minimal of them? want to make sure the test data has same distribution tho....


In [16]:
# Define a dictionary mapping generic color names to their potential variations
color_map = {
    'white': ['white', 'snow', 'ivory', 'pearl', 'cream', 'frost', 'glacier', 'ice', 'chalk', 'yulong'],
    'black': ['black', 'ebony', 'onyx', 'jet', 'noir', 'raven', 'nightfall', 'nero', 'noctis', 'moonlight', 'obsidian'],
    'blue': ['blue', 'navy', 'sapphire', 'indigo', 'caelum', 'reflex', 'sea', 'tempest', 'blu'],
    'red': ['red', 'crimson', 'scarlet', 'ruby', 'maroon', 'sangria', 'mars', 'corsa', 'rosso'],
    'green': ['green', 'olive', 'emerald', 'jade', 'lime', 'jungle', 'moss', 'caviar', 'verde'],
    'yellow': ['yellow', 'gold', 'lemon', 'amber', 'hellayella'],
    'silver': ['silver', 'platinum', 'steel', 'zynith', 'radiance', 'metallic', 'magno'], # we mayyy want to remove metallic from silver...
    'purple': ['purple', 'lavender', 'amethyst', 'violet', 'plum'],
    'gray': ['gray', 'grey', 'charcoal', 'slate', 'graphite', 'ash'],
    'orange': ['orange', 'tangerine', 'apricot', 'peach', 'mango'],
    'brown': ['brown', 'tan', 'chocolate', 'camel', 'khaki', 'dune'],
    'beige': ['beige', 'cream', 'vanilla', 'linen', 'isis', 'lunar'],
    'other': ['pink', 'c / c', '–', 'custom'] #another way to deal with '–' is to put it as another category, but we will try this
}

def transform_color(color_str):
    color_str = color_str.lower()
    for generic_color, variations in color_map.items():
        for variation in variations:
            if variation in color_str:
                return generic_color.capitalize()
    return color_str

# Apply the transform_color function to the 'Color' column
train_df2 = train_df.copy()
train_df2['ext_col'] = train_df2['ext_col'].apply(transform_color)
print("train ext_col:", train_df2.ext_col.unique())

test_df2 = test_df.copy()
test_df2['ext_col'] = test_df2['ext_col'].apply(transform_color)
print("test ext_col:", test_df2.ext_col.unique())

train ext_col: ['Blue' 'Black' 'Purple' 'Gray' 'White' 'Red' 'Silver' 'Green' 'Orange'
 'Beige' 'Yellow' 'Brown' 'Other']
test ext_col: ['White' 'Silver' 'Blue' 'Red' 'Black' 'Gray' 'Green' 'Purple' 'Other'
 'Orange' 'Yellow' 'Brown' 'Beige']


In [17]:
# print(" pink in ext_col:", train_df2[train_df2.ext_col == 'pink'].shape[0])
# print(" c / c in ext_col:", train_df2[train_df2.ext_col == 'c / c'].shape[0])
# print(" custom color in ext_col:", train_df2[train_df2.ext_col == 'custom color'].shape[0])

# Remove rows with edge cases
# train_df2 = train_df2[train_df2['ext_col'] != 'pink']
# train_df2 = train_df2[train_df2['ext_col'] != "c / c"]
# train_df2 = train_df2[train_df2['ext_col'] != 'custom color']
# print("ext_col:", train_df2.ext_col.unique())
print(train_df2.shape)
print(test_df2.shape)

(54273, 13)
(36183, 12)


In [18]:
# print(" pink in ext_col:", test_df2[test_df2.ext_col == 'pink'].shape[0])
# print(" c / c in ext_col:", test_df2[test_df2.ext_col == 'c / c'].shape[0])
# print(" custom color in ext_col:", test_df2[test_df2.ext_col == 'custom color'].shape[0])

# # Remove rows with edge cases
# test_df2 = test_df2[test_df2['ext_col'] != 'pink']
# test_df2 = test_df2[test_df2['ext_col'] != "c / c"]
# test_df2 = test_df2[test_df2['ext_col'] != 'custom color']
# print("ext_col:", test_df2.ext_col.unique())
# test_df2.shape

In [19]:
# drop some useless columns
drop_col = ['model', 'engine', 'transmission', 'int_col', 'clean_title']
train_df2 = train_df2.drop(drop_col, axis=1)
train_df2.head(10)

,id,brand,model_year,milage,fuel_type,ext_col,accident,price
0,0,Ford,2018,74349,Gasoline,Blue,0,11000
1,1,BMW,2007,80000,Gasoline,Black,0,8250
2,2,Jaguar,2009,91491,Gasoline,Purple,0,15000
3,3,BMW,2022,2437,Hybrid,Gray,0,63500
4,4,Pontiac,2001,111000,Gasoline,White,0,7850
5,5,Acura,2003,124756,Gasoline,Red,1,4995
6,6,Audi,2014,107380,Gasoline,Gray,0,26500
7,7,GMC,2019,51300,Gasoline,White,1,25500
8,8,Audi,2016,87842,Gasoline,Silver,0,13999
9,9,Acura,2007,152270,Gasoline,Gray,1,6700


In [20]:
test_df2 = test_df2.drop(drop_col, axis=1)
test_df2.head(10)

,id,brand,model_year,milage,fuel_type,ext_col,accident
0,54273,Mercedes-Benz,2014,73000,Gasoline,White,0
1,54274,Lexus,2015,128032,Gasoline,Silver,0
2,54275,Mercedes-Benz,2015,51983,Gasoline,Blue,0
3,54276,Land,2018,29500,Gasoline,White,1
4,54277,BMW,2020,90000,Gasoline,White,1
5,54278,Chevrolet,2018,2894,Gasoline,Silver,0
6,54279,Land,2019,41200,Gasoline,Silver,1
7,54280,Land,2019,58000,Gasoline,White,1
8,54281,Dodge,2013,124705,Gasoline,Red,1
9,54282,Audi,2022,29850,Gasoline,Black,0


### 2.2.3 ~One-hot~ Encoding the categorical data
there is actually quite an amount of color type (12) and even more brand... we need to do other types of encoding:

* label encoding: using unique integer value to encode (can introduce unwanted relationship)
* frequent/ infrequent encoding: we can group the infrequent categories into single cat to prevent high cardinality, lets look into this first

as for fuel_type, we will perform one-hot encoding since there is only 3 unique values

In [21]:
# see the unique value counts of the high cardinality columns
print(train_df2['brand'].value_counts())
print(train_df2['ext_col'].value_counts())

brand
BMW              7369
Ford             6706
Mercedes-Benz    5087
Chevrolet        4424
Audi             2922
Porsche          2627
Toyota           2315
Lexus            2257
Jeep             2247
Land             1995
Cadillac         1565
Nissan           1252
GMC              1076
Dodge            1011
RAM               966
INFINITI          957
Lincoln           767
Mazda             748
Subaru            739
Hyundai           694
Jaguar            661
Volkswagen        628
Honda             624
Acura             580
Kia               526
Volvo             452
MINI              364
Maserati          293
Bentley           284
Chrysler          258
Genesis           249
Buick             228
Mitsubishi        182
Hummer            176
Pontiac           149
Alfa              144
Rolls-Royce       142
Lamborghini       122
Tesla             110
Ferrari            87
Saturn             58
Scion              53
Aston              50
McLaren            43
Rivian             27
Merc

seeing how many relatively rare values there are, we can suggest:

* Grouping brand of less than 500 to 'Others', reducing 53 to 26
* Grouping ext_col of less than 1000 to 'Other", reducing 12 to 7

In [22]:
other_color = ['Yellow', 'Green', 'Beige', 'Brown', 'Orange', 'Purple']
other_brand = ['Volvo', 'MINI', 'Maserati', 'Bentley', 'Genesis', 'Chrysler', 'Buick', 'Mitsubishi', 'Hummer', 'Pontiac', 'Alfa', 
               'Rolls-Royce', 'Lamborghini', 'Tesla', 'Ferrari', 'Saturn', 'Scion', 'Aston', 'McLaren', 'Rivian', 'FIAT',
               'Lotus', 'Mercury', 'Suzuki', 'Maybach', 'Lucid', 'Plymouth', 'Bugatti', 'Saab', 'smart']

# Using replace function to group edge cases of brand & ext_col
train_df3 = train_df2.copy()
train_df3['ext_col'] = train_df3['ext_col'].replace(other_color, 'Other')
train_df3['brand'] = train_df3['brand'].replace(other_brand, 'Other')

test_df3 = test_df2.copy()
test_df3['ext_col'] = test_df3['ext_col'].replace(other_color, 'Other')
test_df3['brand'] = test_df3['brand'].replace(other_brand, 'Other')

In [23]:
print(train_df3['brand'].value_counts())
print(train_df3['ext_col'].value_counts())

brand
BMW              7369
Ford             6706
Mercedes-Benz    5087
Chevrolet        4424
Other            3530
Audi             2922
Porsche          2627
Toyota           2315
Lexus            2257
Jeep             2247
Land             1995
Cadillac         1565
Nissan           1252
GMC              1076
Dodge            1011
RAM               966
INFINITI          957
Lincoln           767
Mazda             748
Subaru            739
Hyundai           694
Jaguar            661
Volkswagen        628
Honda             624
Acura             580
Kia               526
Name: count, dtype: int64
ext_col
Black     15723
White     14072
Gray       7938
Silver     5593
Blue       4795
Red        3217
Other      2935
Name: count, dtype: int64


In [24]:
print(test_df3['brand'].value_counts())
print(test_df3['ext_col'].value_counts())

brand
BMW              4854
Ford             4382
Mercedes-Benz    3265
Chevrolet        2993
Other            2371
Audi             1919
Porsche          1805
Toyota           1574
Lexus            1530
Jeep             1510
Land             1360
Cadillac         1059
Nissan            922
GMC               749
Dodge             705
INFINITI          666
RAM               606
Mazda             528
Lincoln           496
Subaru            492
Jaguar            447
Hyundai           436
Honda             405
Acura             386
Volkswagen        376
Kia               347
Name: count, dtype: int64
ext_col
Black     10432
White      9235
Gray       5399
Silver     3728
Blue       3242
Red        2183
Other      1964
Name: count, dtype: int64


In [25]:
# Define your own encoding dictionary
ext_col_encoding = {
    'Black': 0,
    'White': 1,
    'Gray': 2,
    'Silver': 3,
    'Blue': 4,
    'Red': 5,
    'Other': 6    
}

brand_encoding = {
    'BMW': 0, 
    'Ford': 1,
    'Mercedes-Benz': 2,
    'Chevrolet': 3,
    'Audi': 4,
    'Porsche': 5,
    'Toyota': 6,
    'Lexus': 7,  
    'Jeep': 8,  
    'Land': 9,  
    'Cadillac': 10,  
    'Nissan': 11,  
    'GMC': 12,  
    'INFINITI': 13,  
    'Dodge': 14,  
    'RAM': 15,  
    'Mazda': 16,  
    'Lincoln': 17,  
    'Subaru': 18,  
    'Jaguar': 19,  
    'Hyundai': 20,  
    'Honda': 21,  
    'Acura': 22,  
    'Volkswagen': 23,  
    'Kia': 24,  
    'Other': 25
}


# Encode the 'ext_col' column using the predefined dictionary
train_df3['ext_col_encoded'] = train_df3['ext_col'].map(ext_col_encoding)
train_df3['brand_encoded'] = train_df3['brand'].map(brand_encoding)

# One-hot encode fuel_type
train_df3 = pd.get_dummies(train_df3, columns=['fuel_type'], dtype='int')

train_df3.head()

,id,brand,model_year,milage,ext_col,accident,price,ext_col_encoded,brand_encoded,fuel_type_Diesel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_–
0,0,Ford,2018,74349,Blue,0,11000,4,1,0,1,0,0
1,1,BMW,2007,80000,Black,0,8250,0,0,0,1,0,0
2,2,Jaguar,2009,91491,Other,0,15000,6,19,0,1,0,0
3,3,BMW,2022,2437,Gray,0,63500,2,0,0,0,1,0
4,4,Other,2001,111000,White,0,7850,1,25,0,1,0,0


In [26]:
# Encode the 'ext_col' column using the predefined dictionary
test_df3['ext_col_encoded'] = test_df3['ext_col'].map(ext_col_encoding)
test_df3['brand_encoded'] = test_df3['brand'].map(brand_encoding)

# One-hot encode fuel_type
test_df3 = pd.get_dummies(test_df3, columns=['fuel_type'], dtype='int')

test_df3.head()

,id,brand,model_year,milage,ext_col,accident,ext_col_encoded,brand_encoded,fuel_type_Diesel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_–
0,54273,Mercedes-Benz,2014,73000,White,0,1,2,0,1,0,0
1,54274,Lexus,2015,128032,Silver,0,3,7,0,1,0,0
2,54275,Mercedes-Benz,2015,51983,Blue,0,4,2,0,1,0,0
3,54276,Land,2018,29500,White,1,1,9,0,1,0,0
4,54277,BMW,2020,90000,White,1,1,0,0,1,0,0


### Feature Engineering: adding a new column of car age & miles_per_year
car_age = current_year - mileage
miles_per_year = milage / car_age

In [27]:
# Feature engineering
current_year = 2024.5  # Update this to the current year
train_df3['car_age'] = current_year - train_df3['model_year']
train_df3['miles_per_year'] = train_df3['milage'] / train_df3['car_age']

In [28]:
test_df3['car_age'] = current_year - test_df3['model_year']
test_df3['miles_per_year'] = test_df3['milage'] / test_df3['car_age']

### 2.2.4 Make a new dataframe for the preprocessed data

In [29]:
train_df = train_df3[['id', 'model_year', 'car_age', 'milage', 'miles_per_year', 'fuel_type_Diesel', 'fuel_type_Gasoline', 'fuel_type_Hybrid', 'fuel_type_–', 'accident', 'brand_encoded', 
                      'ext_col_encoded', 'price']].copy()
train_df.head()

,id,model_year,car_age,milage,miles_per_year,fuel_type_Diesel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_–,accident,brand_encoded,ext_col_encoded,price
0,0,2018,6.5,74349,11438.307692,0,1,0,0,0,1,4,11000
1,1,2007,17.5,80000,4571.428571,0,1,0,0,0,0,0,8250
2,2,2009,15.5,91491,5902.645161,0,1,0,0,0,19,6,15000
3,3,2022,2.5,2437,974.800000,0,0,1,0,0,0,2,63500
4,4,2001,23.5,111000,4723.404255,0,1,0,0,0,25,1,7850


In [30]:
test_df = test_df3[['id', 'model_year', 'car_age', 'milage', 'miles_per_year', 'fuel_type_Diesel', 'fuel_type_Gasoline', 'fuel_type_Hybrid', 'fuel_type_–',  
                    'accident', 'brand_encoded', 'ext_col_encoded']].copy()
test_df.head()

,id,model_year,car_age,milage,miles_per_year,fuel_type_Diesel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_–,accident,brand_encoded,ext_col_encoded
0,54273,2014,10.5,73000,6952.380952,0,1,0,0,0,2,1
1,54274,2015,9.5,128032,13477.052632,0,1,0,0,0,7,3
2,54275,2015,9.5,51983,5471.894737,0,1,0,0,0,2,4
3,54276,2018,6.5,29500,4538.461538,0,1,0,0,1,9,1
4,54277,2020,4.5,90000,20000.000000,0,1,0,0,1,0,1


In [31]:
train_df.describe()

,id,model_year,car_age,milage,miles_per_year,fuel_type_Diesel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_–,brand_encoded,ext_col_encoded,price
count,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,5.427300e+04
mean,27136.000000,2015.091979,9.408021,72746.175667,8109.411459,0.020434,0.938183,0.035893,0.005491,7.553369,1.835204,3.921844e+04
std,15667.409917,5.588909,5.588909,50469.490448,5232.943285,0.141480,0.240825,0.186024,0.073897,7.664878,1.801713,7.282634e+04
min,0.000000,1974.000000,0.500000,100.000000,5.897436,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000e+03
25%,13568.000000,2012.000000,5.500000,32268.000000,5036.363636,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.550000e+04
50%,27136.000000,2016.000000,8.500000,66107.000000,7612.903226,0.000000,1.000000,0.000000,0.000000,5.000000,1.000000,2.800000e+04
75%,40704.000000,2019.000000,12.500000,102000.000000,10322.580645,0.000000,1.000000,0.000000,0.000000,11.000000,3.000000,4.500000e+04
max,54272.000000,2024.000000,50.500000,405000.000000,167756.000000,1.000000,1.000000,1.000000,1.000000,25.000000,6.000000,2.954083e+06


In [32]:
print("brand_encoded:", train_df.brand_encoded.unique())
print("brand_encoded:", test_df.brand_encoded.unique())

brand_encoded: [ 1  0 19 25 22  4 12  3  5  2  7 24 17 14 23  9 10 16 15 18 20  8 21 11
  6 13]
brand_encoded: [ 2  7  9  0  3 14  4  1 24  6 10 12  8 16 22 13 23 18 20 19  5 17 11 15
 25 21]


In [33]:
print("ext_col_encoded:", train_df.ext_col_encoded.unique())
print("ext_col_encoded:", test_df.ext_col_encoded.unique())

ext_col_encoded: [4 0 6 2 1 5 3]
ext_col_encoded: [1 3 4 5 0 2 6]


converting data type int to float:

In [34]:
train_df['accident'] = train_df['accident'].astype(int)
train_df['price'] = train_df['price'].astype(float)
train_df['fuel_type_Diesel'] = train_df['fuel_type_Diesel'].astype(float)
train_df['fuel_type_Gasoline'] = train_df['fuel_type_Gasoline'].astype(float)
train_df['fuel_type_Hybrid'] = train_df['fuel_type_Hybrid'].astype(float)
train_df.dtypes

id                      int64
model_year              int64
car_age               float64
milage                  int64
miles_per_year        float64
fuel_type_Diesel      float64
fuel_type_Gasoline    float64
fuel_type_Hybrid      float64
fuel_type_–             int64
accident                int64
brand_encoded           int64
ext_col_encoded         int64
price                 float64
dtype: object

In [35]:
test_df['accident'] = test_df['accident'].astype(int)
test_df['fuel_type_Diesel'] = test_df['fuel_type_Diesel'].astype(float)
test_df['fuel_type_Gasoline'] = test_df['fuel_type_Gasoline'].astype(float)
test_df['fuel_type_Hybrid'] = test_df['fuel_type_Hybrid'].astype(float)
test_df.dtypes

id                      int64
model_year              int64
car_age               float64
milage                  int64
miles_per_year        float64
fuel_type_Diesel      float64
fuel_type_Gasoline    float64
fuel_type_Hybrid      float64
fuel_type_–             int64
accident                int64
brand_encoded           int64
ext_col_encoded         int64
dtype: object

# 3. Build a baseline model
We will build a basic model using XGBoost Regressor as a baseline here ~where our second, more advanced model (deep learning?) will try to beat.~

codes adapted from ["Accelerating XGBoost with GPU" post](https://www.kaggle.com/code/vinhnguyen/accelerating-xgboost-with-gpu), had to modify for regression usecase (instead of the original classification) 

refering to [early stopping](https://mljar.com/blog/xgboost-early-stopping/)...

In [36]:
import cupy as cp
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import xgboost as xgb
import numpy as np
from sklearn.feature_selection import SelectFromModel

# Define the learning rate scheduler function
def lr_scheduler(num_boost_round):
    def learning_rate(boosting_round):
        base_lr = 0.1
        decay_factor = 0.99
        return base_lr * (decay_factor ** boosting_round)
    return learning_rate

target = 'price'
predictors = [col for col in train_df.columns if col != 'price' and col != 'id']

# Feature selection
selector = SelectFromModel(xgb.XGBRegressor(n_estimators=100, random_state=2019))
selector.fit(train_df[predictors], train_df[target])
predictors = train_df[predictors].columns[selector.get_support()].tolist()

print("predictors are:",  predictors)

# Split the data into training and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=2019)

# Feature scaling (only for continuous variables)
continuous_features = ['milage', 'model_year', 'car_age', 'miles_per_year']
scaler = StandardScaler()
train_df[continuous_features] = scaler.fit_transform(train_df[continuous_features])
val_df[continuous_features] = scaler.transform(val_df[continuous_features])
test_df[continuous_features] = scaler.transform(test_df[continuous_features])

predictors are: ['model_year', 'milage', 'miles_per_year', 'accident', 'brand_encoded', 'ext_col_encoded']


# 4. Model Training & Tuning
First, we are going to perform a 5 fold randomized CV for the xgb model

Then, with the obtained best parameters, we will train the full training data (training+validation) on the final model

After that, predictions on test data will be generated using the model.

In [37]:
%%time

# Define the hyperparameter search space
param_dist = {
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'gamma': [0, 0.1, 0.3, 0.5, 0.7, 0.9],
    'min_child_weight': [1, 3, 5, 7],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'n_estimators': [100, 200, 300, 400, 500],
    'reg_alpha': [0.1, 0.5, 0.75, 1],
    'reg_lambda': [0.1, 0.5, 0.75, 1]
}


# Create XGBRegressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', tree_method='hist', device='cuda')

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    xgb_model, 
    param_distributions=param_dist, 
    n_iter=100, 
    cv=5, 
    scoring='neg_root_mean_squared_error', 
    random_state=2019, 
    n_jobs=-1, 
    verbose=2
)

# Fit RandomizedSearchCV
random_search.fit(train_df[predictors], train_df[target])

# Get the best model
best_model = random_search.best_estimator_

# Make predictions on validation set
val_predictions = best_model.predict(val_df[predictors])
val_rmse = np.sqrt(metrics.mean_squared_error(val_df[target], val_predictions))
print(f"Validation RMSE: {val_rmse:.6f}")

# Make predictions on test set
test_predictions = best_model.predict(test_df[predictors])
# test_rmse = np.sqrt(metrics.mean_squared_error(test_df[target], test_predictions))
print(f"Test predictions: ", test_predictions)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:09:23] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:09:23] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for boo

[CV] END colsample_bytree=0.9, gamma=0.5, learning_rate=0.15, max_depth=6, min_child_weight=3, n_estimators=100, reg_alpha=0.1, reg_lambda=0.75, subsample=0.6; total time=   1.9s
[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_depth=6, min_child_weight=3, n_estimators=100, reg_alpha=1, reg_lambda=0.1, subsample=0.8; total time=   1.4s
[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_depth=6, min_child_weight=3, n_estimators=100, reg_alpha=1, reg_lambda=0.1, subsample=0.8; total time=   1.4s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=5, min_child_weight=3, n_estimators=200, reg_alpha=0.5, reg_lambda=1, subsample=0.9; total time=   2.5s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.05, max_depth=7, min_child_weight=3, n_estimators=400, reg_alpha=0.75, reg_lambda=0.5, subsample=0.7; total time=   6.3s
[CV] END colsample_bytree=0.8, gamma=0.9, learning_rate=0.01, max_depth=7, min_child_weight=3, n_estimators=400, reg_

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:16:34] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


validation rmse > cv rmse, sign of overfitting?

In [38]:
val_df['predicted_price'] = val_predictions
val_df.head()

,id,model_year,car_age,milage,miles_per_year,fuel_type_Diesel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_–,accident,brand_encoded,ext_col_encoded,price,predicted_price
23504,23504,0.160663,-0.160663,0.518330,0.678007,0.0,1.0,0.0,0,1,23,4,25900.0,19137.072266
41858,41858,0.699103,-0.699103,-1.402327,-1.489410,0.0,1.0,0.0,0,0,25,4,39000.0,102367.054688
46003,46003,1.237544,-1.237544,-0.809953,0.893096,0.0,1.0,0.0,0,0,0,2,115785.0,57618.574219
7698,7698,0.160663,-0.160663,-0.550902,-0.542321,0.0,1.0,0.0,0,0,21,1,23000.0,32213.601562
22340,22340,-1.813618,1.813618,0.066282,-0.808933,0.0,1.0,0.0,0,1,6,2,19500.0,22961.160156


In [39]:
#confirm if prediction size match that of the test data
test_predictions.size

36183

# 5. Generating final output

In [40]:
final_pred = test_df[['id']].copy()
final_pred['prediction'] = test_predictions
final_pred.head(20)

,id,prediction
0,54273,27082.445312
1,54274,21630.005859
2,54275,40519.910156
3,54276,44039.515625
4,54277,40118.257812
5,54278,102512.078125
6,54279,41060.968750
7,54280,36868.945312
8,54281,14831.613281
9,54282,67217.328125


In [41]:
# final_pred.to_csv('/kaggle/working/submission1.csv', index=False)
# final_pred.to_csv('/kaggle/working/submission2.csv', index=False)
# final_pred.to_csv('/kaggle/working/submission3.csv', index=False)
# final_pred.to_csv('/kaggle/working/submission4.csv', index=False)
final_pred.to_csv('/kaggle/working/submission5.csv', index=False)

can track feature importance with XGBoost's model.feature_importances_

we are at place 500+, but loss is in range with top performers. next optimisation we can do:

two to try first:

* ~check the official model params~

* Bin continuous variables

* (-) Try different scaling methods for your features, such as StandardScaler or MinMaxScaler

others::

* monitor loss to implement early stopping? already have early stopping in xgb.train, need to plot the loss to monitor.
* Create interaction features
* Create polynomial features

16/6 things to try: 
- ~rerun once with higher epoch + scaler, do a submission 4 see how~ (overfitting a lot, turning down the number of iterations)
- ~min max scaler leads to lower performance, why? overfitting? try another normalisation? or should we normalise actual continuous number? (instead of ALL predictors)~
- early stopping parameter in xgb.train, what does it mean? seeing the validation is doing poorer than training, should we stop early?
- ~check if there is redundancy in the 2 CVs...~
- hyperparam tuning using hyperopt, ref stackoverflow [post](https://stackoverflow.com/questions/66463422/xgboost-parameter-tuning-random-search)


###### tips to try:

1. Feature engineering:
* Create interaction terms, especially between continuous and categorical variables.
* Add polynomial features for continuous variables.
* ~Create a 'car_age' feature by subtracting 'model_year' from the current year.~


2. Handle outliers:

* Check for and handle outliers in continuous variables like 'price' and 'mileage'.


3. ~Feature selection:~

* ~Use feature importance from XGBoost to select the most important features.~


4. Separate validation set:

* Consider creating a separate validation set instead of using the test set for final evaluation.


5. Hyperparameter tuning:

* Consider using Bayesian Optimization instead of RandomizedSearchCV for more efficient hyperparameter tuning.


6. Ensemble methods:

* Try stacking with other models like LightGBM or CatBoost.


7. ~Learning rate scheduling:~

* ~Implement a learning rate scheduler to reduce the learning rate over time.~

# Final Prediction